<a href="https://colab.research.google.com/github/jbastianrj/Asset-allocation-w-several-proximity-measures/blob/main/master_tesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación y carga de librerías

In [1]:
! pip install skfolio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.2/691.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.0 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.3.4
    Uninstalling cvxpy-1.3.4:
      Successfully uninstalled cvxpy-1.3.4


In [2]:
! pip install dtw-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 5.4 MB/s eta 0:00:00


In [3]:
from plotly.io import show
from sklearn.model_selection import train_test_split

from skfolio import Population, RiskMeasure
from skfolio.cluster import HierarchicalClustering, LinkageMethod
#from skfolio.datasets import load_sp500_dataset
from skfolio.distance import (
    CovarianceDistance,
    DistanceCorrelation,
    KendallDistance,
    MutualInformation,
    NBinsMethod,
    PearsonDistance,
    SpearmanDistance,
)
from skfolio.optimization import (
    EqualWeighted,
    HierarchicalRiskParity,
    HierarchicalEqualRiskContribution,
    MeanRisk, ObjectiveFunction,
)
from skfolio.preprocessing import prices_to_returns

In [4]:
import numpy as np
import pandas as pd

In [5]:
import scipy.spatial.distance as ssd
from skfolio.datasets import load_sp500_dataset
from skfolio.preprocessing import prices_to_returns
from sklearn.model_selection import train_test_split
from scipy.stats import skew, kurtosis

from dtw import dtw
from joblib import Parallel, delayed
from scipy.spatial.distance import pdist, squareform

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [6]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [7]:
from skfolio import Portfolio

# Carga de datos

In [8]:
# Cargamos los datos y obtención de rendimientos
prices = load_sp500_dataset()

SP = prices_to_returns(prices)*100  # Para expresarlo en términos de porcentajes
SP_train, SP_test = train_test_split(SP, test_size=0.2, shuffle=False, random_state=42)
# Independientemente de el random siempre escoge el 80% iniciales para train y 20 para test

In [9]:
SP_train.head() # Contiene fechas del 1990-ENERO-03 al 2016-MAYO-20

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
1990-01-03,0.757576,-3.030303,0.804523,11.805556,-1.622921,-0.187617,0.358102,0.407213,3.358869,-1.431767,0.000000,1.589561,0.520833,-0.970874,0.293830,-0.181347,0.0,-1.935484,0.000000,-1.007866
1990-01-04,0.375940,-1.550000,-2.135462,-1.242236,-1.283096,-0.563910,0.624442,0.202781,0.399088,-0.499319,-0.555722,-1.564689,2.849741,-0.980392,1.660156,-1.972489,0.0,-0.986842,-0.520120,-0.993295
1990-01-05,0.374532,-3.199594,-2.182059,0.000000,-1.485455,-0.945180,-1.329787,-1.040763,0.397501,-0.821168,-1.087449,-2.064057,-2.518892,-1.399053,-0.864553,-1.800371,0.0,-4.318937,-1.073198,-0.526712
1990-01-08,0.373134,0.000000,0.563317,-7.547170,0.942408,0.572519,-0.988320,1.694420,0.000000,2.115915,0.000000,1.283915,1.550388,1.811831,-0.872093,1.833378,0.0,-2.083333,1.363004,1.538074
1990-01-09,-0.743494,1.652676,0.000000,0.000000,-0.746888,-2.080259,-2.631579,-3.102557,-3.195701,-0.765766,-1.114674,-0.789285,-0.254453,-1.372213,-2.150538,0.000000,0.0,-2.482270,-2.661910,-2.011423


# Cálculo de medidas de proximidad

## Normalizar distancias

In [10]:
# Función que normaliza las matrices (min-max)

def normalizar_matriz(matriz):
    """
    Normaliza una matriz al rango de 0 a 1 utilizando la técnica min-max.

    Parámetros:
    - matriz: np.ndarray
        La matriz que se desea normalizar.

    Retorna:
    - np.ndarray
        La matriz normalizada.
    """
    # Calcula los valores mínimo y máximo de la matriz
    min_valor = np.min(matriz)
    max_valor = np.max(matriz)

    # Normaliza la matriz al rango de 0 a 1
    matriz_normalizada = (matriz - min_valor) / (max_valor - min_valor)

    return np.around(matriz_normalizada, decimals=6)

## Conversión a una matriz triangular condensada útil para cálculo de dendrogramas.



In [11]:
# Función que condensa matriz cuadrada a una matriz triangular condensada

def condensar_matriz(matriz):
    """
    Convierte una matriz cuadrada a una matriz triangular condensada útil para cálculo de dendrogramas.

    Parámetros:
    - matriz: np.ndarray
        La matriz que se desea convertir.

    Retorna:
    - np.ndarray
        La matriz condensada.
    """
    # Calcula los valores mínimo y máximo de la matriz
    matriz_condensada = ssd.squareform(matriz)

    return matriz_condensada

## Correlación $D_1$ $=$ $1-\rho$

In [12]:
# Función que calcula la matriz de proximidad basada en correlación

def correlacion(df, corrcoef = 'pearson'):
    """
    Calcula la matriz de proximidad basada en correlación
    entre series de tiempo a partir de un dataframe

    Parámetros:
    - df: pd.DataFrame
        Es el dataframe que contiene las series de tiempo
        de las que se desea calcular la matriz de proximidad
    - corrcoef: str
        Es la correlación a utilizar: Pearson, Kendall, Spearman

    Retorna:
    - np.ndarray
        La matriz de proximidad calculada.
    """
    # Calcula la correlación indicada y posteriormente calcula la matriz de proximidad
    pearson = df.corr(corrcoef)
    matriz_calculada = 1 - pearson

    return matriz_calculada

# Esta distancia fue sugerida por asesores como una correlación simple

In [13]:
# Ejemplos de uso
correlacion(SP_train)

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,0.695574,0.758481,0.784425,0.815582,0.690415,0.723931,0.838820,0.724896,0.826787,0.844124,0.820094,0.638428,0.862397,0.828572,0.845615,0.886148,0.829751,0.774052,0.804741
AMD,0.695574,0.000000,0.738170,0.788217,0.794974,0.709748,0.723992,0.846298,0.703424,0.851747,0.840159,0.835902,0.676120,0.854370,0.829933,0.882394,0.873044,0.847896,0.830005,0.802194
BAC,0.758481,0.738170,0.000000,0.761051,0.650203,0.460210,0.605199,0.729147,0.281458,0.731087,0.728150,0.721468,0.680637,0.749229,0.690338,0.724804,0.827623,0.730140,0.720461,0.671503
BBY,0.784425,0.788217,0.761051,0.000000,0.843964,0.704051,0.634729,0.834458,0.713848,0.847435,0.827807,0.834634,0.735033,0.849661,0.824674,0.845179,0.898542,0.855078,0.719036,0.846101
CVX,0.815582,0.794974,0.650203,0.843964,0.000000,0.606977,0.711090,0.682714,0.652444,0.685480,0.721760,0.684517,0.694395,0.708135,0.674493,0.712039,0.679679,0.739972,0.752743,0.224621
GE,0.690415,0.709748,0.460210,0.704051,0.606977,0.000000,0.527724,0.613361,0.435479,0.624309,0.637577,0.630862,0.583042,0.658641,0.591847,0.623241,0.820719,0.715136,0.590766,0.590030
HD,0.723931,0.723992,0.605199,0.634729,0.711090,0.527724,0.000000,0.681909,0.554939,0.657569,0.697794,0.688197,0.607040,0.679918,0.676797,0.693114,0.877645,0.724566,0.469193,0.689834
JNJ,0.838820,0.846298,0.729147,0.834458,0.682714,0.613361,0.681909,0.000000,0.710184,0.583438,0.507668,0.471573,0.697688,0.615840,0.470334,0.567395,0.894894,0.701381,0.658774,0.632849
JPM,0.724896,0.703424,0.281458,0.713848,0.652444,0.435479,0.554939,0.710184,0.000000,0.706779,0.698846,0.679101,0.607731,0.733608,0.669887,0.710574,0.829907,0.729048,0.663661,0.659713
KO,0.826787,0.851747,0.731087,0.847435,0.685480,0.624309,0.657569,0.583438,0.706779,0.000000,0.644081,0.635118,0.708522,0.480300,0.633892,0.543585,0.889635,0.739283,0.648366,0.632800


In [14]:
# Ejemplos de uso
normalizar_matriz(correlacion(SP_train))

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,0.750659,0.818548,0.846547,0.880171,0.745092,0.781262,0.905250,0.782303,0.892263,0.910973,0.885040,0.688987,0.930694,0.894190,0.912582,0.956325,0.895462,0.835352,0.868471
AMD,0.750659,0.000000,0.796628,0.850639,0.857932,0.765955,0.781327,0.913320,0.759131,0.919201,0.906694,0.902100,0.729664,0.922031,0.895658,0.952274,0.942184,0.915044,0.895736,0.865723
BAC,0.818548,0.796628,0.000000,0.821322,0.701695,0.496655,0.653127,0.786891,0.303748,0.788985,0.785815,0.778604,0.734539,0.808563,0.745009,0.782204,0.893165,0.787963,0.777517,0.724682
BBY,0.846547,0.850639,0.821322,0.000000,0.910801,0.759808,0.684996,0.900542,0.770380,0.914546,0.893365,0.900732,0.793243,0.916949,0.889983,0.912112,0.969701,0.922795,0.775979,0.913107
CVX,0.880171,0.857932,0.701695,0.910801,0.000000,0.655046,0.767404,0.736781,0.704113,0.739766,0.778919,0.738727,0.749387,0.764215,0.727909,0.768428,0.733505,0.798573,0.812355,0.242409
GE,0.745092,0.765955,0.496655,0.759808,0.655046,0.000000,0.569517,0.661936,0.469966,0.673751,0.688070,0.680822,0.629216,0.710801,0.638718,0.672598,0.885715,0.771770,0.637551,0.636757
HD,0.781262,0.781327,0.653127,0.684996,0.767404,0.569517,0.000000,0.735912,0.598886,0.709644,0.753054,0.742698,0.655113,0.733763,0.730395,0.748005,0.947149,0.781947,0.506350,0.744464
JNJ,0.905250,0.913320,0.786891,0.900542,0.736781,0.661936,0.735912,0.000000,0.766426,0.629643,0.547872,0.508919,0.752940,0.664611,0.507582,0.612329,0.965765,0.756926,0.710945,0.682966
JPM,0.782303,0.759131,0.303748,0.770380,0.704113,0.469966,0.598886,0.766426,0.000000,0.762752,0.754190,0.732881,0.655859,0.791705,0.722938,0.766847,0.895631,0.786785,0.716219,0.711958
KO,0.892263,0.919201,0.788985,0.914546,0.739766,0.673751,0.709644,0.629643,0.762752,0.000000,0.695088,0.685415,0.764632,0.518337,0.684093,0.586633,0.960089,0.797829,0.699712,0.682914


## Correlacción (*correlation distance o standar angular distance*) $D_2 =\sqrt{\frac{1}{2}(1-\rho)}$

In [15]:
# Función que calcula la matriz de proximidad de standar angular distance

def standar_angular(df, corrcoef = 'pearson'):
    """
    Calcula la matriz de proximidad basada la standar angular
    distance entre series de tiempo a partir de un dataframe

    Parámetros:
    - df: pd.DataFrame
        Es el dataframe que contiene las series de tiempo
        de las que se desea calcular la matriz de proximidad
    - corrcoef: str
        Es la correlación a utilizar: Pearson, Kendall, Spearman

    Retorna:
    - np.ndarray
        La matriz de proximidad calculada.
    """
    # Calcula la correlación indicada y posteriormente calcula la matriz de proximidad
    pearson = df.corr(corrcoef)
    matriz_calculada = np.sqrt(0.5*(1 - pearson))

    return matriz_calculada

# "Building Diversified Portfolios that Outperform Out-of-Sample",
# Lòpez de Prado, Journal of Portfolio Management (2016) (Primera distancia del apendice)
# Esta coincide con la implementada con distance de PearsonDistance de skfolio
# Es realmente una métrica

In [16]:
# Ejemplos de uso
standar_angular(SP_train)

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,0.589735,0.615825,0.626269,0.638585,0.587544,0.601636,0.647619,0.602037,0.642957,0.649663,0.640349,0.564990,0.656657,0.643651,0.650236,0.665638,0.644108,0.622114,0.634327
AMD,0.589735,0.000000,0.607524,0.627781,0.630466,0.595713,0.601661,0.650499,0.593053,0.652590,0.648135,0.646491,0.581429,0.653594,0.644179,0.664227,0.660698,0.651113,0.644207,0.633322
BAC,0.615825,0.607524,0.000000,0.616868,0.570177,0.479692,0.550090,0.603799,0.375139,0.604602,0.603386,0.600612,0.583368,0.612058,0.587511,0.601998,0.643282,0.604210,0.600192,0.579441
BBY,0.626269,0.627781,0.616868,0.000000,0.649601,0.593318,0.563351,0.645933,0.597431,0.650936,0.643354,0.646001,0.606231,0.651790,0.642135,0.650069,0.670277,0.653865,0.599598,0.650423
CVX,0.638585,0.630466,0.570177,0.649601,0.000000,0.550898,0.596276,0.584258,0.571158,0.585440,0.600733,0.585029,0.589235,0.595036,0.580729,0.596674,0.582958,0.608265,0.613491,0.335128
GE,0.587544,0.595713,0.479692,0.593318,0.550898,0.000000,0.513675,0.553788,0.466626,0.558708,0.564614,0.561632,0.539927,0.573864,0.543989,0.558230,0.640593,0.597970,0.543492,0.543153
HD,0.601636,0.601661,0.550090,0.563351,0.596276,0.513675,0.000000,0.583913,0.526754,0.573397,0.590675,0.586599,0.550926,0.583060,0.581720,0.588691,0.662437,0.601899,0.484351,0.587296
JNJ,0.647619,0.650499,0.603799,0.645933,0.584258,0.553788,0.583913,0.000000,0.595896,0.540110,0.503819,0.485579,0.590630,0.554906,0.484940,0.532632,0.668915,0.592191,0.573922,0.562516
JPM,0.602037,0.593053,0.375139,0.597431,0.571158,0.466626,0.526754,0.595896,0.000000,0.594466,0.591120,0.582709,0.551240,0.605643,0.578743,0.596060,0.644169,0.603758,0.576047,0.574331
KO,0.642957,0.652590,0.604602,0.650936,0.585440,0.558708,0.573397,0.540110,0.594466,0.000000,0.567486,0.563524,0.595198,0.490051,0.562980,0.521337,0.666946,0.607981,0.569371,0.562494


## Correlación (*Absolute angular distance*) $D_3 = \sqrt{1-|\rho|}$

In [17]:
# Función que calcula la matriz de proximidad de absolute angular distance

def absolute_angular(df, corrcoef = 'pearson'):
    """
    Calcula la matriz de proximidad basada la absolute angular
    distance entre series de tiempo a partir de un dataframe

    Parámetros:
    - df: pd.DataFrame
        Es el dataframe que contiene las series de tiempo
        de las que se desea calcular la matriz de proximidad
    - corrcoef: str
        Es la correlación a utilizar: Pearson, Kendall, Spearman

    Retorna:
    - np.ndarray
        La matriz de proximidad calculada.
    """
    # Calcula la correlación indicada y posteriormente calcula la matriz de proximidad
    pearson = df.corr(corrcoef)
    absoluto = np.abs(pearson)
    matriz_calculada = np.sqrt(1 - absoluto)

    return matriz_calculada

# "Building Diversified Portfolios that Outperform Out-of-Sample",
# López de Prado, Journal of Portfolio Management (2016) (Segunda distancia del apendice)
# Es realmente una métrica

In [18]:
# Ejemplos de uso
absolute_angular(SP_train)

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,0.834011,0.870908,0.885678,0.903096,0.830912,0.850841,0.915871,0.851408,0.909278,0.918762,0.905590,0.799017,0.928653,0.910260,0.919573,0.941354,0.910906,0.879802,0.897073
AMD,0.834011,0.000000,0.859168,0.887816,0.891613,0.842465,0.850877,0.919945,0.838704,0.922902,0.916602,0.914277,0.822265,0.924322,0.911006,0.939358,0.934368,0.920813,0.911046,0.895653
BAC,0.870908,0.859168,0.000000,0.872383,0.806352,0.678387,0.777945,0.853901,0.530526,0.855036,0.853317,0.849393,0.825007,0.865580,0.830866,0.851354,0.909738,0.854482,0.848800,0.819453
BBY,0.885678,0.887816,0.872383,0.000000,0.918675,0.839078,0.796699,0.913487,0.844895,0.920562,0.909839,0.913583,0.857341,0.921771,0.908116,0.919336,0.947914,0.924704,0.847960,0.919837
CVX,0.903096,0.891613,0.806352,0.918675,0.000000,0.779087,0.843261,0.826265,0.807740,0.827937,0.849565,0.827355,0.833304,0.841508,0.821275,0.843824,0.824427,0.860216,0.867607,0.473942
GE,0.830912,0.842465,0.678387,0.839078,0.779087,0.000000,0.726446,0.783174,0.659908,0.790133,0.798484,0.794268,0.763572,0.811567,0.769316,0.789456,0.905935,0.845657,0.768613,0.768134
HD,0.850841,0.850877,0.777945,0.796699,0.843261,0.726446,0.000000,0.825778,0.744942,0.810906,0.835340,0.829576,0.779127,0.824571,0.822677,0.832535,0.936827,0.851214,0.684976,0.830562
JNJ,0.915871,0.919945,0.853901,0.913487,0.826265,0.783174,0.825778,0.000000,0.842724,0.763831,0.712508,0.686712,0.835277,0.784755,0.685809,0.753256,0.945989,0.837485,0.811649,0.795518
JPM,0.851408,0.838704,0.530526,0.844895,0.807740,0.659908,0.744942,0.842724,0.000000,0.840702,0.835970,0.824076,0.779571,0.856509,0.818466,0.842956,0.910992,0.853843,0.814654,0.812227
KO,0.909278,0.922902,0.855036,0.920562,0.827937,0.790133,0.810906,0.763831,0.840702,0.000000,0.802546,0.796943,0.841737,0.693037,0.796174,0.737282,0.943205,0.859815,0.805212,0.795487


## Mantegna $D_4$ = $\sqrt{2(1-\rho)}$

In [19]:
# Función que calcula la matriz de proximidad de mantegna

def mantegna(df, corrcoef = 'pearson'):
    """
    Calcula la matriz de proximidad basada en mantegna
    distance entre series de tiempo a partir de un dataframe

    Parámetros:
    - df: pd.DataFrame
        Es el dataframe que contiene las series de tiempo
        de las que se desea calcular la matriz de proximidad
    - corrcoef: str
        Es la correlación a utilizar: Pearson, Kendall, Spearman

    Retorna:
    - np.ndarray
        La matriz de proximidad calculada.
    """
    # Calcula la correlación indicada y posteriormente calcula la matriz de proximidad
    pearson = df.corr(corrcoef)
    matriz_calculada = np.sqrt(2*(1 - pearson))

    return matriz_calculada

# Utilizada en HCAA Raffinot

In [20]:
# Ejemplos de uso
mantegna(SP_train)

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,1.179469,1.231650,1.252538,1.277170,1.175087,1.203272,1.295238,1.204073,1.285913,1.299326,1.280698,1.129980,1.313314,1.287301,1.300473,1.331276,1.288216,1.244228,1.268653
AMD,1.179469,0.000000,1.215047,1.255561,1.260932,1.191426,1.203322,1.300998,1.186107,1.305180,1.296270,1.292982,1.162858,1.307188,1.288358,1.328453,1.321397,1.302226,1.288414,1.266645
BAC,1.231650,1.215047,0.000000,1.233735,1.140353,0.959385,1.100181,1.207599,0.750278,1.209204,1.206773,1.201223,1.166736,1.224115,1.175022,1.203997,1.286563,1.208421,1.200384,1.158882
BBY,1.252538,1.255561,1.233735,0.000000,1.299203,1.186635,1.126702,1.291865,1.194862,1.301871,1.286707,1.292001,1.212463,1.303581,1.284269,1.300137,1.340553,1.307729,1.199196,1.300847
CVX,1.277170,1.260932,1.140353,1.299203,0.000000,1.101796,1.192552,1.168515,1.142317,1.170880,1.201466,1.170057,1.178470,1.190071,1.161458,1.193347,1.165915,1.216530,1.226982,0.670255
GE,1.175087,1.191426,0.959385,1.186635,1.101796,0.000000,1.027350,1.107575,0.933251,1.117416,1.129228,1.123265,1.079854,1.147729,1.087977,1.116459,1.281186,1.195940,1.086983,1.086306
HD,1.203272,1.203322,1.100181,1.126702,1.192552,1.027350,0.000000,1.167826,1.053507,1.146795,1.181350,1.173198,1.101853,1.166120,1.163441,1.177382,1.324873,1.203799,0.968703,1.174592
JNJ,1.295238,1.300998,1.207599,1.291865,1.168515,1.107575,1.167826,0.000000,1.191792,1.080221,1.007639,0.971157,1.181260,1.109811,0.969880,1.065265,1.337830,1.184382,1.147845,1.125032
JPM,1.204073,1.186107,0.750278,1.194862,1.142317,0.933251,1.053507,1.191792,0.000000,1.188932,1.182240,1.165419,1.102480,1.211287,1.157486,1.192119,1.288338,1.207517,1.152094,1.148662
KO,1.285913,1.305180,1.209204,1.301871,1.170880,1.117416,1.146795,1.080221,1.188932,0.000000,1.134972,1.127047,1.190396,0.980102,1.125960,1.042674,1.333893,1.215963,1.138741,1.124989


Al normalizar standar, angular y mantegna se obtiene la misma matriz de proximidad o similaridad

In [21]:
# Ejemplos de uso
normalizar_matriz(standar_angular(SP_train))

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,0.866406,0.904737,0.920080,0.938174,0.863187,0.883890,0.951446,0.884479,0.944597,0.954449,0.940766,0.830053,0.964725,0.945616,0.955292,0.977919,0.946288,0.913976,0.931918
AMD,0.866406,0.000000,0.892540,0.922301,0.926246,0.875189,0.883927,0.955678,0.871281,0.958749,0.952205,0.949789,0.854204,0.960225,0.946392,0.975845,0.970662,0.956579,0.946433,0.930443
BAC,0.904737,0.892540,0.000000,0.906268,0.837672,0.704738,0.808163,0.887069,0.551133,0.888248,0.886462,0.882386,0.857052,0.899202,0.863139,0.884423,0.945074,0.887673,0.881769,0.851283
BBY,0.920080,0.922301,0.906268,0.000000,0.954359,0.871670,0.827645,0.948969,0.877713,0.956319,0.945180,0.949069,0.890642,0.957575,0.943389,0.955045,0.984734,0.960622,0.880897,0.955566
CVX,0.938174,0.926246,0.837672,0.954359,0.000000,0.809349,0.876016,0.858359,0.839115,0.860096,0.882564,0.859492,0.865671,0.874194,0.853176,0.876600,0.856449,0.893629,0.901308,0.492351
GE,0.863187,0.875189,0.704738,0.871670,0.809349,0.000000,0.754664,0.813594,0.685541,0.820823,0.829500,0.825119,0.793231,0.843090,0.799198,0.820120,0.941124,0.878505,0.798468,0.797971
HD,0.883890,0.883927,0.808163,0.827645,0.876016,0.754664,0.000000,0.857853,0.773877,0.842404,0.867787,0.861799,0.809391,0.856600,0.854632,0.864873,0.973216,0.884278,0.711583,0.862823
JNJ,0.951446,0.955678,0.887069,0.948969,0.858359,0.813594,0.857853,0.000000,0.875457,0.793501,0.740184,0.713386,0.867721,0.815237,0.712448,0.782514,0.982733,0.870015,0.843175,0.826418
JPM,0.884479,0.871281,0.551133,0.877713,0.839115,0.685541,0.773877,0.875457,0.000000,0.873357,0.868441,0.856085,0.809851,0.889778,0.850258,0.875698,0.946378,0.887009,0.846297,0.843776
KO,0.944597,0.958749,0.888248,0.956319,0.860096,0.820823,0.842404,0.793501,0.873357,0.000000,0.833719,0.827898,0.874432,0.719956,0.827099,0.765920,0.979841,0.893213,0.836488,0.826386


In [22]:
# Ejemplos de uso
normalizar_matriz(absolute_angular(SP_train))

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,0.866406,0.904737,0.920080,0.938174,0.863187,0.883890,0.951446,0.884479,0.944597,0.954449,0.940766,0.830053,0.964725,0.945616,0.955292,0.977919,0.946288,0.913976,0.931918
AMD,0.866406,0.000000,0.892540,0.922301,0.926246,0.875189,0.883927,0.955678,0.871281,0.958749,0.952205,0.949789,0.854204,0.960225,0.946392,0.975845,0.970662,0.956579,0.946433,0.930443
BAC,0.904737,0.892540,0.000000,0.906268,0.837672,0.704738,0.808163,0.887069,0.551133,0.888248,0.886462,0.882386,0.857052,0.899202,0.863139,0.884423,0.945074,0.887673,0.881769,0.851283
BBY,0.920080,0.922301,0.906268,0.000000,0.954359,0.871670,0.827645,0.948969,0.877713,0.956319,0.945180,0.949069,0.890642,0.957575,0.943389,0.955045,0.984734,0.960622,0.880897,0.955566
CVX,0.938174,0.926246,0.837672,0.954359,0.000000,0.809349,0.876016,0.858359,0.839115,0.860096,0.882564,0.859492,0.865671,0.874194,0.853176,0.876600,0.856449,0.893629,0.901308,0.492351
GE,0.863187,0.875189,0.704738,0.871670,0.809349,0.000000,0.754664,0.813594,0.685541,0.820823,0.829500,0.825119,0.793231,0.843090,0.799198,0.820120,0.941124,0.878505,0.798468,0.797971
HD,0.883890,0.883927,0.808163,0.827645,0.876016,0.754664,0.000000,0.857853,0.773877,0.842404,0.867787,0.861799,0.809391,0.856600,0.854632,0.864873,0.973216,0.884278,0.711583,0.862823
JNJ,0.951446,0.955678,0.887069,0.948969,0.858359,0.813594,0.857853,0.000000,0.875457,0.793501,0.740184,0.713386,0.867721,0.815237,0.712448,0.782514,0.982733,0.870015,0.843175,0.826418
JPM,0.884479,0.871281,0.551133,0.877713,0.839115,0.685541,0.773877,0.875457,0.000000,0.873357,0.868441,0.856085,0.809851,0.889778,0.850258,0.875698,0.946378,0.887009,0.846297,0.843776
KO,0.944597,0.958749,0.888248,0.956319,0.860096,0.820823,0.842404,0.793501,0.873357,0.000000,0.833719,0.827898,0.874432,0.719956,0.827099,0.765920,0.979841,0.893213,0.836488,0.826386


In [23]:
# Ejemplos de uso
normalizar_matriz(mantegna(SP_train))

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,0.866406,0.904737,0.920080,0.938174,0.863187,0.883890,0.951446,0.884479,0.944597,0.954449,0.940766,0.830053,0.964725,0.945616,0.955292,0.977919,0.946288,0.913976,0.931918
AMD,0.866406,0.000000,0.892540,0.922301,0.926246,0.875189,0.883927,0.955678,0.871281,0.958749,0.952205,0.949789,0.854204,0.960225,0.946392,0.975845,0.970662,0.956579,0.946433,0.930443
BAC,0.904737,0.892540,0.000000,0.906268,0.837672,0.704738,0.808163,0.887069,0.551133,0.888248,0.886462,0.882386,0.857052,0.899202,0.863139,0.884423,0.945074,0.887673,0.881769,0.851283
BBY,0.920080,0.922301,0.906268,0.000000,0.954359,0.871670,0.827645,0.948969,0.877713,0.956319,0.945180,0.949069,0.890642,0.957575,0.943389,0.955045,0.984734,0.960622,0.880897,0.955566
CVX,0.938174,0.926246,0.837672,0.954359,0.000000,0.809349,0.876016,0.858359,0.839115,0.860096,0.882564,0.859492,0.865671,0.874194,0.853176,0.876600,0.856449,0.893629,0.901308,0.492351
GE,0.863187,0.875189,0.704738,0.871670,0.809349,0.000000,0.754664,0.813594,0.685541,0.820823,0.829500,0.825119,0.793231,0.843090,0.799198,0.820120,0.941124,0.878505,0.798468,0.797971
HD,0.883890,0.883927,0.808163,0.827645,0.876016,0.754664,0.000000,0.857853,0.773877,0.842404,0.867787,0.861799,0.809391,0.856600,0.854632,0.864873,0.973216,0.884278,0.711583,0.862823
JNJ,0.951446,0.955678,0.887069,0.948969,0.858359,0.813594,0.857853,0.000000,0.875457,0.793501,0.740184,0.713386,0.867721,0.815237,0.712448,0.782514,0.982733,0.870015,0.843175,0.826418
JPM,0.884479,0.871281,0.551133,0.877713,0.839115,0.685541,0.773877,0.875457,0.000000,0.873357,0.868441,0.856085,0.809851,0.889778,0.850258,0.875698,0.946378,0.887009,0.846297,0.843776
KO,0.944597,0.958749,0.888248,0.956319,0.860096,0.820823,0.842404,0.793501,0.873357,0.000000,0.833719,0.827898,0.874432,0.719956,0.827099,0.765920,0.979841,0.893213,0.836488,0.826386


## Dynamic Time Warping DTW

In [24]:
# Función para calcular la distancia DTW entre dos series temporales

def calculate_dtw_distance(series1, series2, method = 'euclidean', symmetric = 1):
    """
    Calcula la distancia DTW entre dos series temporales

    Parámetros:
    - series1: np.ndarray
        Contiene la primera serie de tiempo
    - series2: np.ndarray
        Contiene la segunda serie de tiempo
    - method: str
        Indica el método con el que se cálcula la matriz inicial
    - symmetric: int
        step_pattern = 1: Identica con dtw de R
        step_pattern = 2: Identica con DTWARP de TSclust

    Retorna:
    - dist: float
        La distancia DTW entre dos series temporales
    """
    # Calculamos la secuencia de deformación usando el método euclideano y el step_pattern indicado
    alignment = dtw(series1.T, series2.T, dist_method = method, step_pattern=f"symmetric{symmetric}")
    # Obtenemos la distancia acumulada del patrón obtenido
    dist = alignment.distance

    return dist


    # step_pattern = 1: Identica con dtw de R
    # step_pattern = 2: Identica con DTWARP de TSclust

In [25]:
# Definimos una función que realice el cálculo individual de fila por columna

def calculate_distance(i, j, df, assets, method, symmetric):
    asset1 = df[assets[i]].values
    asset2 = df[assets[j]].values
    distance = calculate_dtw_distance(asset1, asset2, method, symmetric)  # Se usó DTWARP
    return i, j, distance

In [26]:
# Definimos la función que permite reducir tiempo de ejecución paralelizando y aprovechando simetría

def parallel_dtw_distance_matrix(df, method = 'euclidean', symmetric = 1):
    """
    Calcula la distancia DTW entre dos series temporales

    Parámetros:
    - df: pd.DataFrame
        Es el dataframe que contiene las series de tiempo
        de las que se desea calcular la matriz de proximidad
    - method: str
        Indica el método con el que se cálcula la matriz inicial
    - symmetric: int
        step_pattern = 1: Identica con dtw de R
        step_pattern = 2: Identica con DTWARP de TSclust

    Retorna:
    - distance_matrix: np.ndarray
        La matriz de proximidad calculada.
    """

    assets = df.columns
    num_assets = len(assets)
    distance_matrix = np.zeros((num_assets, num_assets))

    # Crear las combinaciones únicas (i, j) donde i <= j
    indices = [(i, j) for i in range(num_assets) for j in range(i, num_assets)]

    # Ejecutar en paralelo
    results = Parallel(n_jobs=-1)(delayed(calculate_distance)(i, j, df, assets, method, symmetric) for i, j in indices)

    # Rellenar la matriz de distancias usando los resultados
    for i, j, distance in results:
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance  # Aprovechar la simetría

    # Asignamos los nombres de activos a los ejes
    distance_matrix = pd.DataFrame(distance_matrix)
    distance_matrix.index = assets
    distance_matrix = distance_matrix.set_axis(assets, axis="columns")

    return distance_matrix


In [27]:
# Ejemplos de uso
dtw_matrix = parallel_dtw_distance_matrix(SP_train)

In [28]:
dtw_matrix

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,12970.065949,10740.294503,11404.881338,9413.535817,9527.710820,9339.984007,9469.598836,10175.008266,9366.400420,9380.510808,9378.269439,9314.095146,9350.357613,9315.916125,9345.544169,12302.895119,10009.633931,9192.700850,9357.402503
AMD,12970.065949,0.000000,13228.037697,13546.479449,13086.407560,12858.728318,12686.963433,13389.380582,12757.832177,13212.847591,12989.635628,12978.395233,12634.395763,13238.201482,12894.576672,13321.439992,14236.741926,12989.956775,12993.613429,13154.074875
BAC,10740.294503,13228.037697,0.000000,11539.806040,7824.277796,7408.002059,8343.076470,7975.289397,7090.399621,7821.709387,8291.060380,8150.050482,8534.475414,7976.391168,8149.044520,7851.933170,12155.046286,9103.042832,8226.386647,7739.727755
BBY,11404.881338,13546.479449,11539.806040,0.000000,10685.935647,10646.044439,10184.386867,10675.539582,10885.712605,10624.893639,10503.146922,10647.824674,10378.211050,10542.850981,10435.199597,10652.106195,13094.878615,10852.137542,10382.960836,10677.724465
CVX,9413.535817,13086.407560,7824.277796,10685.935647,0.000000,5839.883448,6686.135404,5534.742866,7655.231429,5599.425197,6060.848108,5960.953024,6931.620849,5808.177686,6062.918499,5495.496005,11211.640085,7725.051992,6171.536115,4268.332888
GE,9527.710820,12858.728318,7408.002059,10646.044439,5839.883448,0.000000,6659.201617,5864.065183,7191.735678,5753.638224,6392.670325,6194.403789,6871.875696,6025.367924,6192.597683,5780.791863,11583.780151,7854.097223,6266.466576,5693.063929
HD,9339.984007,12686.963433,8343.076470,10184.386867,6686.135404,6659.201617,0.000000,6534.514105,7878.281886,6482.146776,6912.294049,6908.343532,7229.314953,6570.916012,6784.984376,6533.758051,11631.576095,8128.891520,6448.856304,6603.590207
JNJ,9469.598836,13389.380582,7975.289397,10675.539582,5534.742866,5864.065183,6534.514105,0.000000,7722.620933,4865.525678,5534.316172,5430.171683,6673.987890,5066.779369,5547.651917,4808.512416,11818.722287,7529.022557,5588.376121,5238.628798
JPM,10175.008266,12757.832177,7090.399621,10885.712605,7655.231429,7191.735678,7878.281886,7722.620933,0.000000,7599.731811,7955.619262,7887.385158,8035.229228,7737.968176,7774.294145,7679.081315,11799.724896,8745.334869,7872.849425,7572.462874
KO,9366.400420,13212.847591,7821.709387,10624.893639,5599.425197,5753.638224,6482.146776,4865.525678,7599.731811,0.000000,5773.852897,5697.909868,6650.219730,4897.638392,5805.820541,4856.067877,11662.454967,7569.052883,5597.872410,5290.495087


In [29]:
normalizar_matriz(dtw_matrix)

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,0.911028,0.754407,0.801088,0.661214,0.669234,0.656048,0.665152,0.714701,0.657903,0.658894,0.658737,0.654229,0.656777,0.654357,0.656438,0.864165,0.703085,0.645703,0.657271
AMD,0.911028,0.000000,0.929148,0.951515,0.919200,0.903207,0.891142,0.940481,0.896120,0.928081,0.912402,0.911613,0.887450,0.929862,0.905725,0.935708,1.000000,0.912425,0.912682,0.923953
BAC,0.754407,0.929148,0.000000,0.810565,0.549583,0.520344,0.586024,0.560191,0.498035,0.549403,0.582371,0.572466,0.599468,0.560268,0.572395,0.551526,0.853780,0.639405,0.577828,0.543645
BBY,0.801088,0.951515,0.810565,0.000000,0.750589,0.747787,0.715359,0.749858,0.764621,0.746301,0.737749,0.747912,0.728974,0.740538,0.732977,0.748212,0.919795,0.762263,0.729307,0.750012
CVX,0.661214,0.919200,0.549583,0.750589,0.000000,0.410198,0.469639,0.388765,0.537710,0.393308,0.425719,0.418702,0.486883,0.407971,0.425864,0.386008,0.787514,0.542614,0.433494,0.299811
GE,0.669234,0.903207,0.520344,0.747787,0.410198,0.000000,0.467748,0.411897,0.505153,0.404140,0.449026,0.435100,0.482686,0.423227,0.434973,0.406047,0.813654,0.551678,0.440162,0.399885
HD,0.656048,0.891142,0.586024,0.715359,0.469639,0.467748,0.000000,0.458989,0.553377,0.455311,0.485525,0.485248,0.507793,0.461546,0.476583,0.458936,0.817011,0.570980,0.452973,0.463841
JNJ,0.665152,0.940481,0.560191,0.749858,0.388765,0.411897,0.458989,0.000000,0.542443,0.341758,0.388735,0.381420,0.468786,0.355895,0.389671,0.337754,0.830156,0.528844,0.392532,0.367965
JPM,0.714701,0.896120,0.498035,0.764621,0.537710,0.505153,0.553377,0.542443,0.000000,0.533811,0.558809,0.554016,0.564401,0.543521,0.546073,0.539385,0.828822,0.614279,0.552995,0.531896
KO,0.657903,0.928081,0.549403,0.746301,0.393308,0.404140,0.455311,0.341758,0.533811,0.000000,0.405560,0.400226,0.467117,0.344014,0.407805,0.341094,0.819180,0.531656,0.393199,0.371609


## Distancia Euclidiana

In [30]:
# Función que calcula la matriz de proximidad basada en correlación

def euclidiana(df):
    """
    Calcula la matriz de proximidad basada en la distancia euclidiana
    entre series de tiempo a partir de un dataframe

    Parámetros:
    - df: pd.DataFrame
        Es el dataframe que contiene las series de tiempo
        de las que se desea calcular la matriz de proximidad

    Retorna:
    - np.ndarray
        La matriz de proximidad calculada.
    """
    assets = df.columns
    # Calcula la correlación indicada y posteriormente calcula la matriz de proximidad
    matriz_calculada = pdist(df.T, metric='euclidean')
    # Convertimos el condensado a matriz cuadrada
    matriz_calculada = squareform(matriz_calculada)

    # Asignamos los nombres de activos a los ejes
    matriz_calculada = pd.DataFrame(matriz_calculada)
    matriz_calculada.index = assets
    matriz_calculada = matriz_calculada.set_axis(assets, axis="columns")

    return matriz_calculada

# Esta distancia fue sugerida por asesores como una correlación simple

In [31]:
# Ejemplos de uso
euclidiana(SP_train)

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
AAPL,0.000000,336.904736,282.926426,319.301973,247.318454,236.367359,248.419497,244.962226,265.413330,245.457360,256.925718,253.114332,235.769255,251.331834,253.848406,247.194902,367.241095,280.853394,245.354233,243.917994
AMD,336.904736,0.000000,340.290441,374.820645,321.411278,312.783554,319.038597,324.528536,325.600582,326.352854,331.169556,330.148736,311.641748,327.854118,329.072518,329.607161,417.263728,351.178682,328.167626,321.107855
BAC,282.926426,340.290441,0.000000,306.951098,214.036017,188.783321,218.352035,220.037821,160.162110,221.728455,229.344501,227.859392,232.099770,225.392332,223.217344,220.686404,348.163478,254.001974,226.183768,215.290593
BBY,319.301973,374.820645,306.951098,0.000000,281.201925,267.222812,261.027171,275.801679,288.040644,278.799885,284.202803,284.595000,278.864233,280.568776,282.915564,278.219164,389.953760,311.152475,267.457425,279.547995
CVX,247.318454,321.411278,214.036017,281.201925,0.000000,150.893869,176.970609,140.213402,197.542577,143.607375,163.033046,157.794553,177.470589,148.963580,156.052413,145.696701,294.121376,204.713205,162.438984,83.397629
GE,236.367359,312.783554,188.783321,267.222812,150.893869,0.000000,160.297848,144.962494,169.362846,148.598566,162.858748,161.132968,170.520641,154.951055,155.586324,147.976504,317.013098,208.639676,153.599202,145.941887
HD,248.419497,319.038597,218.352035,261.027171,176.970609,160.297848,0.000000,167.281573,195.627623,166.670929,182.950217,180.915995,183.751203,171.366574,178.964604,170.294669,332.017360,219.194231,148.396502,171.677378
JNJ,244.962226,324.528536,220.037821,275.801679,140.213402,144.962494,167.281573,0.000000,200.168750,123.899296,131.266579,125.753443,171.693493,130.494816,124.985794,121.517552,317.341187,195.087182,144.287985,130.983118
JPM,265.413330,325.600582,160.162110,288.040644,197.542577,169.362846,195.627623,200.168750,0.000000,201.430985,209.218253,205.881401,205.877976,206.294695,204.253227,201.530880,339.033510,240.545279,202.132575,196.651538
KO,245.457360,326.352854,221.728455,278.799885,143.607375,148.598566,166.670929,123.899296,201.430985,0.000000,149.491108,147.418539,175.120944,117.944938,146.660934,121.972990,318.282179,201.199691,145.905688,134.121948


# Implementación de HCAA

## Funciones para obtener levels, indices y pesos asignados usando HCAA

In [32]:
# Clases para Obtener índices de agrupación y asignaciones de peso

# Auxiliary function to get the node_id and the capital allocation for each cluster
def get_levels(n_clusters: int, n_filas: int, mat_Z: np.matrix):
    nodos_id = []
    nivel_nodo = []
    filas_Z = mat_Z.shape[0]
    current_bif = mat_Z[-1]
    nodos_id.append(int(current_bif[0]))
    nivel_nodo.append(.5)
    nodos_id.append(int(current_bif[1]))
    nivel_nodo.append(.5)
    current_level = 2
    while(current_level < n_clusters):
        id_cluster = filas_Z+n_filas-current_level
        current_bif = mat_Z[-current_level]
        index_of_id = nodos_id.index(id_cluster)
        new_weight = nivel_nodo[index_of_id] / 2
        del nivel_nodo[index_of_id]
        del nodos_id[index_of_id]
        nodos_id.append(int(current_bif[0]))
        nivel_nodo.append(new_weight)
        nodos_id.append(int(current_bif[1]))
        nivel_nodo.append(new_weight)
        current_level += 1
    return(nodos_id, nivel_nodo)

# Auxiliary function to get the assets id's of each cluster
def get_leaves(id_nodo, mat_Z, n_filas):
    if id_nodo <= n_filas -1:
        return [id_nodo]
    fila_Z = mat_Z[id_nodo - n_filas]
    id_nodo_1 = int(fila_Z[0])
    id_nodo_2 = int(fila_Z[1])
    el_1 =0
    el_2 = 0
    if (id_nodo_1 <= n_filas-1) and(id_nodo_2 <= n_filas-1):
        return([id_nodo_1, id_nodo_2])
    elif (id_nodo_1 > n_filas-1)and(id_nodo_2 > n_filas-1):
        el_1 = get_leaves(id_nodo_1, mat_Z, n_filas)
        el_2 = get_leaves(id_nodo_2, mat_Z, n_filas)
        return(el_1 + el_2)
    elif (id_nodo_1 > n_filas-1):
        el_1 = get_leaves(id_nodo_1, mat_Z, n_filas)
        el_1.append(id_nodo_2)
        return(el_1)
    else:
        el_2 = get_leaves(id_nodo_2, mat_Z, n_filas)
        el_2.append(id_nodo_1)
        return(el_2)

# function to the appropiate number of groups given a cutoff point
def get_groups(Z_mat: np.matrix, cutoff_point: float)->int:
    distances = Z_mat[:,2]
    number_groups = int(distances.shape[0] - (sum(distances <= cutoff_point)) +1)
    return number_groups


# function to get the weight of the cluster.
def hcaa_alocation(mat_X: np.matrix, n_clusters:int = 0, custom_proximidad = standar_angular, inverse_data = True, cutoff_point: float = None, return_groups: bool = False)->tuple:
    # Convertir matriz de datos en matriz de distancias
    D_matrix = normalizar_matriz(custom_proximidad(mat_X))
    D_condensed = condensar_matriz(D_matrix)
    Z = linkage(D_condensed, 'ward', optimal_ordering = True)
    if cutoff_point:
        n_clusters = get_groups(Z, cutoff_point)
    n_filas = mat_X.shape[1]
    levels = get_levels(n_clusters, n_filas, Z)
    index_asset = []
    capital_all = []
    index_assets_groups = []
    for index, node_id in enumerate(levels[0]):
        if node_id <= n_filas-1:
            index_asset.append(node_id)
            capital_all.append(levels[1][index])
        else:
            assets = get_leaves(node_id, Z, n_filas)
            cluster_weight = levels[1][index]
            asset_weight = round(cluster_weight/len(assets), 6)
            index_assets_groups.append(assets)
            index_asset += assets
            capital_all += [asset_weight] *len(assets)
    if return_groups:
        return (index_asset, capital_all, index_assets_groups)
    return (index_asset, capital_all)

In [127]:
# Ejemplo de uso
# Obtener índices de agrupación y asignaciones de peso
index_assets, weight_assets = hcaa_alocation(SP_train, n_clusters=6, custom_proximidad = standar_angular, inverse_data = True)
print(index_assets, weight_assets)

[19, 4, 16, 8, 2, 5, 9, 13, 15, 17, 14, 11, 7, 10, 12, 0, 1, 18, 6, 3] [0.083333, 0.083333, 0.083333, 0.083333, 0.083333, 0.083333, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.041667, 0.041667, 0.041667, 0.041667, 0.041667, 0.041667]


In [34]:
index_assets, weight_assets = hcaa_alocation(SP_train, n_clusters=6, custom_proximidad = absolute_angular, inverse_data = True)
print(index_assets, weight_assets)

[19, 4, 16, 8, 2, 5, 9, 13, 15, 17, 14, 11, 7, 10, 12, 0, 1, 18, 6, 3] [0.083333, 0.083333, 0.083333, 0.083333, 0.083333, 0.083333, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.041667, 0.041667, 0.041667, 0.041667, 0.041667, 0.041667]


In [35]:
index_assets, weight_assets = hcaa_alocation(SP_train, n_clusters=6, custom_proximidad = euclidiana, inverse_data = True)
print(index_assets, weight_assets)

[16, 1, 3, 14, 11, 10, 7, 15, 9, 13, 19, 4, 6, 18, 5, 12, 17, 2, 8, 0] [0.5, 0.25, 0.125, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.004464, 0.015625, 0.015625, 0.03125]


In [36]:
index_assets, weight_assets = hcaa_alocation(SP_train, n_clusters=8, custom_proximidad = correlacion, inverse_data = True)
print(index_assets, weight_assets)

[12, 0, 1, 2, 8, 5, 18, 6, 3, 9, 13, 15, 17, 14, 11, 7, 10, 19, 4, 16] [0.083333, 0.083333, 0.083333, 0.041667, 0.041667, 0.041667, 0.041667, 0.041667, 0.041667, 0.041667, 0.041667, 0.041667, 0.0625, 0.015625, 0.015625, 0.015625, 0.015625, 0.0625, 0.0625, 0.125]


In [37]:
index_assets, weight_assets = hcaa_alocation(SP_train, n_clusters=6, custom_proximidad = parallel_dtw_distance_matrix, inverse_data = True)
print(index_assets, weight_assets)

[1, 16, 0, 3, 2, 8, 12, 6, 7, 15, 9, 13, 18, 11, 14, 10, 4, 19, 5, 17] [0.25, 0.125, 0.0625, 0.0625, 0.125, 0.125, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857, 0.017857]


## Clase HCAA dentro de BaseHierarchicalOptimization de skfolio

In [57]:
import numpy as np
import numpy.typing as npt
import pandas as pd
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist, squareform

import skfolio.typing as skt
from skfolio.cluster import HierarchicalClustering
from skfolio.distance import BaseDistance, PearsonDistance
from skfolio.measures import ExtraRiskMeasure, RiskMeasure
from skfolio.optimization.cluster.hierarchical._base import BaseHierarchicalOptimization
from skfolio.prior import BasePrior, EmpiricalPrior
from skfolio.utils.tools import check_estimator

class HierarchicalClusteredAssetAllocation(BaseHierarchicalOptimization):
    """Hierarchical Clustered Asset Allocation estimator."""

    def __init__(
        self,
        risk_measure: RiskMeasure | ExtraRiskMeasure = RiskMeasure.VARIANCE,
        prior_estimator: BasePrior | None = None,
        distance_estimator: BaseDistance | None = None,
        hierarchical_clustering_estimator: HierarchicalClustering | None = None,
        min_weights: skt.MultiInput | None = 0.0,
        max_weights: skt.MultiInput | None = 1.0,
        transaction_costs: skt.MultiInput = 0.0,
        management_fees: skt.MultiInput = 0.0,
        previous_weights: skt.MultiInput | None = None,
        portfolio_params: dict | None = None,
    ):
        super().__init__(
            risk_measure=risk_measure,
            prior_estimator=prior_estimator,
            distance_estimator=distance_estimator,
            hierarchical_clustering_estimator=hierarchical_clustering_estimator,
            min_weights=min_weights,
            max_weights=max_weights,
            transaction_costs=transaction_costs,
            management_fees=management_fees,
            previous_weights=previous_weights,
            portfolio_params=portfolio_params,
        )

    def fit(
        self, X: npt.ArrayLike, y: None = None
    ) -> "HierarchicalClusteredAssetAllocation":
        """Fit the Hierarchical Clustered Asset Allocation estimator.

        Parameters
        ----------
        X : array-like of shape (n_observations, n_assets)
           Price returns of the assets.

        y : Ignored
            Not used, present for API consistency by convention.

        Returns
        -------
        self : HierarchicalClusteredAssetAllocation
            Fitted estimator.
        """
        # Validate
        if not isinstance(self.risk_measure, RiskMeasure | ExtraRiskMeasure):
            raise TypeError(
                "`risk_measure` must be of type `RiskMeasure` or `ExtraRiskMeasure`"
            )
        self.prior_estimator_ = check_estimator(
            self.prior_estimator,
            default=EmpiricalPrior(),
            check_type=BasePrior,
        )
        self.distance_estimator_ = check_estimator(
            self.distance_estimator,
            default=PearsonDistance(),
            check_type=BaseDistance,
        )
        self.hierarchical_clustering_estimator_ = check_estimator(
            self.hierarchical_clustering_estimator,
            default=HierarchicalClustering(),
            check_type=HierarchicalClustering,
        )

        # Fit the estimators
        self.prior_estimator_.fit(X, y)
        prior_model = self.prior_estimator_.prior_model_
        returns = prior_model.returns

        # To keep the asset_names
        if isinstance(X, pd.DataFrame):
            returns = pd.DataFrame(returns, columns=X.columns)

        self.distance_estimator_.fit(returns)
        distance = self.distance_estimator_.distance_

        # To keep the asset_names
        if isinstance(X, pd.DataFrame):
            distance = pd.DataFrame(distance, columns=X.columns)

        self.hierarchical_clustering_estimator_.fit(distance)

        n_clusters = self.hierarchical_clustering_estimator_.n_clusters_
        labels = self.hierarchical_clustering_estimator_.labels_
        linkage_matrix = self.hierarchical_clustering_estimator_.linkage_matrix_

        X = self._validate_data(X)
        n_assets = X.shape[1]

        min_weights, max_weights = self._convert_weights_bounds(n_assets=n_assets)

        # Apply the hcaa_alocation function to get asset allocations
        index_asset, capital_all = hcaa_alocation(mat_X= pd.DataFrame(X), n_clusters=n_clusters)

        self.weights_ = np.zeros(n_assets)
        for i, asset_idx in enumerate(index_asset):
            self.weights_[asset_idx] = capital_all[i]

        return self

    # def predict(self, X: npt.ArrayLike) -> np.ndarray:
    #     """Predict the asset weights.

    #     Parameters
    #     ----------
    #     X : array-like of shape (n_observations, n_assets)
    #         Price returns of the assets.

    #     Returns
    #     -------
    #     weights : ndarray of shape (n_assets,)
    #         Predicted asset weights.
    #     """
    #     return self.weights_

    # def plot_dendrogram(self):
    #     """Plot the dendrogram of the hierarchical clustering."""
    #     import matplotlib.pyplot as plt

    #     linkage_matrix = self.hierarchical_clustering_estimator_.linkage_matrix_
    #     plt.figure(figsize=(10, 7))
    #     sch.dendrogram(linkage_matrix, labels=self.feature_names_in_)
    #     plt.title('Hierarchical Clustering Dendrogram')
    #     plt.xlabel('Asset')
    #     plt.ylabel('Distance')
    #     plt.show()


## Asignación de acitvos con distintos modelos

In [117]:
model1 = HierarchicalRiskParity(
    # risk_measure=RiskMeasure.VARIANCE, # Default
    # distance_estimator=PearsonDistance(), # Default
    hierarchical_clustering_estimator=HierarchicalClustering(
        max_clusters=6, # No se usa el parámetro de # de clusters
        linkage_method=LinkageMethod.WARD, # Default
        ),
    # distance_estimator=Corr_2, # No se puede usar
    # distance_estimator=KendallDistance(),
    portfolio_params=dict(name="HRP-Ward-D2"),
)
model1.fit(SP_train)
model1.weights_

array([0.02898201, 0.01494839, 0.01787051, 0.01916545, 0.05947816,
       0.0668875 , 0.03919105, 0.06583499, 0.02753069, 0.08731924,
       0.06651152, 0.067999  , 0.05473708, 0.08231199, 0.04144045,
       0.08911169, 0.01010366, 0.02281643, 0.05676397, 0.0809962 ])

In [118]:
# Dendrograma
fig = model1.hierarchical_clustering_estimator_.plot_dendrogram(heatmap=True)
fig.update_layout(
     title={
        'text': 'Dendrograma y heatmap usando <i>D<sub>2</sub></i> (6 clusters)',
        'x':0.5,  # Centrar el título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis=dict(
        title='Activos'  # Cambiar la etiqueta del eje X
    ),
    yaxis=dict(
        showline=False,  # Ocultar la línea del eje Y
        showticklabels=False,  # Ocultar las etiquetas de los ticks del eje Y
        showgrid=False,  # Ocultar la cuadrícula del eje Y
        title=''  # Eliminar el título del eje Y
    )
)
show(fig)

In [119]:
model2 = HierarchicalEqualRiskContribution(
    risk_measure=RiskMeasure.VARIANCE, # Default
    # distance_estimator=PearsonDistance(absolute=False, power = 1), # Default
    # distance_estimator=Corr_2(), # Coincide con Pearson
    # distance_estimator=DTWDistance(),
    # distance_estimator=CovarianceDistance(),
    hierarchical_clustering_estimator=HierarchicalClustering(
        max_clusters=6,
        linkage_method=LinkageMethod.WARD, # Default
    ),
    portfolio_params=dict(name="HERC-Ward-D2"),
)
model2.fit(SP_train)
model2.weights_

array([0.01351832, 0.00729099, 0.04831247, 0.00941443, 0.17098952,
       0.11316751, 0.02543936, 0.04167484, 0.05870412, 0.04600527,
       0.02532249, 0.02588881, 0.02669769, 0.04208886, 0.02623262,
       0.04694965, 0.02904629, 0.01643789, 0.03684615, 0.18997273])

In [120]:
# Dendrograma
fig = model2.hierarchical_clustering_estimator_.plot_dendrogram(heatmap=True)
fig.update_layout(
     title={
        'text': 'Dendrograma y heatmap usando <i>D<sub>2</sub></i> (6 clusters)',
        'x':0.5,  # Centrar el título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis=dict(
        title='Activos'  # Cambiar la etiqueta del eje X
    ),
    yaxis=dict(
        showline=False,  # Ocultar la línea del eje Y
        showticklabels=False,  # Ocultar las etiquetas de los ticks del eje Y
        showgrid=False,  # Ocultar la cuadrícula del eje Y
        title=''  # Eliminar el título del eje Y
    )
)
show(fig)

In [126]:
model3 = HierarchicalClusteredAssetAllocation(
    risk_measure=RiskMeasure.VARIANCE, # Default
    # distance_estimator=PearsonDistance(absolute=False, power = 1), # Default
    # distance_estimator=Corr_1(),
    # distance_estimator=DTWDistance(),
    # distance_estimator=CovarianceDistance(),
    hierarchical_clustering_estimator=HierarchicalClustering(
        max_clusters=6,
        linkage_method=LinkageMethod.WARD, # Default
    ),
    portfolio_params=dict(name="HCAA-Ward-D2"),
)

model3.fit(SP_train)
model3.weights_

array([0.041667, 0.041667, 0.083333, 0.041667, 0.083333, 0.083333,
       0.041667, 0.03125 , 0.083333, 0.03125 , 0.03125 , 0.03125 ,
       0.041667, 0.03125 , 0.03125 , 0.03125 , 0.083333, 0.03125 ,
       0.041667, 0.083333])

In [122]:
# Dendograma
fig = model3.hierarchical_clustering_estimator_.plot_dendrogram(heatmap=True)
fig.update_layout(
     title={
        'text': 'Dendrograma y heatmap usando <i>D<sub>1</sub></i> (6 clusters)',
        'x':0.5,  # Centrar el título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis=dict(
        title='Activos'  # Cambiar la etiqueta del eje X
    ),
    yaxis=dict(
        showline=False,  # Ocultar la línea del eje Y
        showticklabels=False,  # Ocultar las etiquetas de los ticks del eje Y
        showgrid=False,  # Ocultar la cuadrícula del eje Y
        title=''  # Eliminar el título del eje Y
    )
)

show(fig)

In [130]:
population_test = Population([])
for model in [model1, model2, model3]:#
              # model5, model6, model7, model8, model9, model10, model11, model12, model13]:
    population_test.append(model.predict(SP_test))
    # population_test.append(model.predict(SP_test))


fig = population_test.plot_cumulative_returns()
fig.update_layout(
     title={
        'text': 'Rendimiento acumulado de distintos portafolios',
        'x':0.5,  # Centrar el título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis=dict(
        title='Activos'  # Cambiar la etiqueta del eje X
    ),
    yaxis=dict(
        title='Rendimientos acumulados'  # Cambiar la etiqueta del eje Y
    ),
    legend = dict(
        title='Portafolios'  # Cambiar la etiqueta del eje Y
    )
)

show(fig)

# Clases para las distintas medidas de proximidad